### 목적 : 미세먼지봇 구현. 
1. 사용자의 텍스트 입력 ex)상도역, 강남 교보문고
2. 구글 API에서 지역 이름 출력(도(광역시), 시, 동(면))
3. 동 이름을 이용하여 환경공단 API에 대입, 최단거리의 관측소 좌표 크롤링
4. 환경공단 API에서 해당 지역 데이터를 제공하지 못하는경우
> 지역이름을 벡터화, 코사인 유사도로 지형적으로 가까운 근처 동 이름 제공
> 관측소 좌표 크롤링  

5. 환경공단 API에서 해당 지역 데이터를 제공하는 경우
> 관측소 좌표 크롤링

6. 관측소를 기준으로 측정한 최신 미세먼지 데이터 크롤링
7. 사용자에게 return


# 4번 단계 구현을 위한 word2vec 과정

####  word2vec의 CBOW
- "비슷한 단어는 같은 문맥에 사용된다"
- text 파일 사용, 성능 향상을 위한 특수 기호 및 영어의 경우 stemming 사용
- 이후 tokenization 수행, 이 때 tokenization은 string -> list를 의미.  
ex) [['token', 'token'], ['token', 'token']]

Codec이란?
- 파이썬에서 일반적으로 유니코드를 다룰 때 사용하는 decode, encode는 대용량 데이터인 경우, 모든 데이터가 메모리에 올라감
- ** codecs ** 모듈을 통해 파일을 조금씩 읽어, 메모리에도 약간의 데이터만 올려줄 수 있음

In [1]:
import codecs
from gensim.models import Word2Vec
import multiprocessing

import pandas as pd
import numpy as np
import glob

C:\Users\ledes\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [1]:
# 주소가 있는 csv 파일들을 불러와 word2vec에 넣기 위한 전처리하는 함수
# 큰 단위의 주소와 세부 단위 주소 따로 합쳐져 있는 파일을 동일 value를 가지는 column을 기준으로 merge
def merge_and_to_csv(add_fname, sub_fname) : 
    address_lines = open("datas/" + add_fname + ".txt", "r")
    sub_lines = open("datas/" + sub_fname + ".txt", "r")
    
    address_list, sub_list = [], []
    
    for add_line in address_lines : 
        add_line = (add_line).split("|")
        address_list.append(add_line)
    
    for sub_line in sub_lines : 
        sub_line = sub_line.split("|")
        sub_list.append(sub_line)
        
    address_df = pd.DataFrame(address_list)
    address_df = address_df[[0, 3, 4, 5]]
    
    sub_df = pd.DataFrame(sub_list)
    sub_df = sub_df[[0, 2, 6, 7]]
    
    full_address = pd.merge(address_df, sub_df, how = 'left', on = [0])
    full_address.to_csv("full_" + add_fname + ".csv", index = False)

In [4]:
#작업 폴더 내 파일들 이름으로 리스트 만들기
ls = glob.glob("C:/Users/ledes/study/Microdust_Kakao_chatbot/chatbot/microdust/datas/*csv")

In [10]:
# 큰 단위 주소, 세부 단위 주소를 인덱싱해 리스트에 담기.
ls = ls[ : -1]
file_ls = []
for idx in range(len(ls)) : 
    f_ls = ls[idx][69 :  ]
    f_ls = f_ls[ : -4]
    file_ls.append(f_ls)
    

In [11]:
for address, sub in zip(file_ls[:17], file_ls[17:]) :
    merge_and_to_csv(address, sub)

----

In [6]:
full_address_ls = []
for name in file_ls[:17] : 
    full_name = "full" + name
    full_address_ls.append(full_name)

In [7]:
#word2vec model들의 이름만 
check_ls = glob.glob("C:/Users/ledes/study/Microdust_Kakao_chatbot/chatbot/microdust/model*")

In [8]:
check_ls = [check_ls[i][69:] for i in range(len(check_ls))]
answer_ls = [file_ls[i][13 : -4] for i in range(len(file_ls))]

In [102]:
#for문 과정에서 발생한 오류 확인 -> 작업이 되지 않은 csv 파일들 내용 확인
result = [i for i in answer_ls if i not in check_ls]
result

['sejong', 'seoul', 'ulsan']

In [143]:
file_ls = glob.glob("C:/Users/ledes/study/Microdust_Kakao_chatbot/chatbot/microdust/*csv")
file_ls = [file_ls[i][63: ] for i in range(len(file_ls))]

In [144]:
file_ls

['full_address_busan.csv',
 'full_address_chungbuk.csv',
 'full_address_chungnam.csv',
 'full_address_daegu.csv',
 'full_address_daejeon.csv',
 'full_address_gangwon.csv',
 'full_address_gwangju.csv',
 'full_address_gyungbuk.csv',
 'full_address_gyunggi.csv',
 'full_address_gyungnam.csv',
 'full_address_incheon.csv',
 'full_address_jeju.csv',
 'full_address_junbuk.csv',
 'full_address_junnam.csv',
 'full_address_sejong.csv',
 'full_address_seoul.csv',
 'full_address_ulsan.csv']

In [150]:
def train_word_vectors(fname) : 
    
    model_name = "_" + fname[13:-4]
    print(model_name + " start")
    #파일 불러오기
    df = pd.read_csv(fname)

    #필요 데이터 추출
    # 세종시 데이터가 기존 칼럼과 달라 오류가 발생 -> 세종시만 따로 처리
    if "sejong" in fname : 
        test.drop(labels = ["0", "4", "6", "7"], inplace = True, axis = 1, errors = "ignore")
    else : 
        df.drop(labels = ["0", "6", "7"], inplace = True, axis = 1, errors = "ignore")

    #null 값 제거
    df.dropna(axis = 0, how = 'any', inplace = True)

    train_data = []
    for idx in np.array(df) : 
        train_data.append(list(idx))

    #word2vec parameters
    parameters = {
        "sg" : 1, # skip-gram
        "size" : 150, # dimesionality of the feature vectors
        "window" : 3,
        "alpha" : 0.01, #learning rate
        "batch_words" : 10000, #사전 구축 시 한 번에 몇 개의 단어를 읽을 것인가
        "iter" : 10, # epoch과 유사
        "workers" : multiprocessing.cpu_count(), #cpu

    }

    #인수 입력
    model = Word2Vec(**parameters)

    #단어 사전 생성
    model.build_vocab(train_data)

    #벡터 train
    model.train(train_data, total_examples = model.corpus_count, epochs = model.epochs)
    
    model.save('model' + model_name)
    
    print(model_name + " end")

In [146]:
for fname in file_ls : 
    train_word_vectors(fname)

_busan start


KeyboardInterrupt: 

In [161]:
fname = "full_address_seoul.csv"
model_name = "_" + fname[13:-4]
print(model_name + " start")
#파일 불러오기
df = pd.read_csv(fname)

#필요 데이터 추출
# 세종시 데이터가 기존 칼럼과 달라 오류가 발생 -> 세종시만 따로 처리
if "sejong" in fname : 
    test.drop(labels = ["0", "4", "6", "7"], inplace = True, axis = 1, errors = "ignore")
else : 
    df.drop(labels = ["0", "6", "7"], inplace = True, axis = 1, errors = "ignore")

#null 값 제거
df.dropna(axis = 0, how = 'any', inplace = True)

train_data = []
for idx in np.array(df) : 
    train_data.append(list(idx))

#word2vec parameters
parameters = {
    "sg" : 1, # skip-gram
    "size" : 150, # dimesionality of the feature vectors
    "window" : 3,
    "alpha" : 0.01, #learning rate
    "batch_words" : 10000, #사전 구축 시 한 번에 몇 개의 단어를 읽을 것인가
    "iter" : 10, # epoch과 유사
    "workers" : multiprocessing.cpu_count(), #cpu

}

#인수 입력
model = Word2Vec(**parameters)

#단어 사전 생성
model.build_vocab(train_data)

#벡터 train
model.train(train_data, total_examples = model.corpus_count, epochs = 100)


print(model_name + " end")

_seoul start
650633 10
_seoul end


In [164]:
model.wv.most_similar("회기동", topn = 10)

[('성수2가제3동', 0.9903359413146973),
 ('장안동', 0.9883288145065308),
 ('성수2가제1동', 0.9877767562866211),
 ('휘경제2동', 0.9877095222473145),
 ('송정동', 0.9870534539222717),
 ('답십리제1동', 0.9870078563690186),
 ('전농제1동', 0.9868310689926147),
 ('성수1가제1동', 0.9866747856140137),
 ('장안제2동', 0.9862525463104248),
 ('장안제1동', 0.9861014485359192)]

In [51]:
model.save('model' + '_seoul')

In [ ]:
model.wv.similarity("흑석동", "도봉구")

In [ ]:
model.wv.most_similar(positive = ["흑석동", "종로구"], negative = ["상도동"])

----

#### 서울을 제외한 다른 데이터들의 학습 정도 확인

In [106]:
file_ls = glob.glob("C:/Users/ledes/study/Microdust_Kakao_chatbot/chatbot/microdust/datas/*txt")
file_ls = [file_ls[idx][69:] for idx in range(len(file_ls))]

In [133]:
with open('datas/' + file_ls[8], 'r') as f :
    test = f.readlines()

In [134]:
ls = []
for idx in range(len(test)) : 
    line = test[idx].split("|")[3:5+1]
    ls.append(line)

In [135]:
train_data = ls

In [138]:
parameters = {
    "sg" : 1, # skip-gram
    "size" : 150, # dimesionality of the feature vectors
    "window" : 3,
    "alpha" : 0.01, #learning rate
    "batch_words" : 10000, #사전 구축 시 한 번에 몇 개의 단어를 읽을 것인가
    "iter" : 10, # epoch과 유사
    "workers" : multiprocessing.cpu_count(), #cpu

}

#인수 입력
model = Word2Vec(**parameters)

#단어 사전 생성
model.build_vocab(train_data)

#벡터 train
model.train(train_data, total_examples = 100, epochs = 100 )

print("end")

end


In [140]:
model.wv.most_similar("삼성동", topn = 10)

[('대치동', 0.9999366402626038),
 ('청담동', 0.9997897148132324),
 ('개포동', 0.9995765686035156),
 ('논현동', 0.9987701177597046),
 ('도곡동', 0.9984256625175476),
 ('역삼동', 0.9982285499572754),
 ('일원동', 0.9977246522903442),
 ('자곡동', 0.9930808544158936),
 ('세곡동', 0.9929624795913696),
 ('율현동', 0.9924757480621338)]